In [1]:
#### This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/2021-ai-w5-p1/train.csv
/kaggle/input/2021-ai-w5-p1/test.csv
/kaggle/input/2021-ai-w5-p1/sample_submit.csv


In [2]:
import torch
import torch.optim as optim

In [3]:
torch.manual_seed(1)

In [4]:
train_data=pd.read_csv("/kaggle/input/2021-ai-w5-p1/train.csv")
test_data=pd.read_csv("/kaggle/input/2021-ai-w5-p1/test.csv")
submit_data=pd.read_csv("/kaggle/input/2021-ai-w5-p1/sample_submit.csv")
print(train_data.shape)
print(test_data.shape)
print(submit_data.shape)

(60000, 785)
(10000, 784)
(10000, 2)


In [5]:
print(train_data.info())
print(test_data.info())
# print(submit_data.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60000 entries, 0 to 59999
Columns: 785 entries, label to 28x28
dtypes: int64(785)
memory usage: 359.3 MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Columns: 784 entries, 1x1 to 28x28
dtypes: int64(784)
memory usage: 59.8 MB
None


In [6]:
xtrain=torch.FloatTensor(train_data.iloc[:,1:].values).cuda()
ytrain=torch.LongTensor(train_data['label'].values).cuda()
print(xtrain.shape)
print(ytrain.shape)

torch.Size([60000, 784])
torch.Size([60000])


In [7]:
from torch.utils.data import TensorDataset # 텐서데이터셋
from torch.utils.data import DataLoader # 데이터로더

In [8]:
dataset = TensorDataset(xtrain, ytrain)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

In [9]:
import torch.nn.functional as F  # for softmax

# 모델 초기화
nb_class = 10
nb_data = len(ytrain)

W = torch.zeros((784, nb_class), requires_grad=True, device="cuda")
b = torch.zeros(1, requires_grad=True, device="cuda")

# optimizer 설정
optimizer = optim.Adam([W, b], lr=0.00001)
nb_epochs = 10
for epoch in range(nb_epochs + 1):
    for batch_idx, samples in enumerate(dataloader):
        x_train, y_train = samples
        hypothesis = F.softmax(x_train.matmul(W) + b, dim=1) # or .mm or @
        y_one_hot=F.one_hot(y_train, num_classes=10)
        
#         y_one_hot = torch.zeros(nb_data, nb_class).cuda()
#         y_one_hot.scatter_(1, y_train.unsqueeze(1), 1)
        cost = (y_one_hot * -torch.log(F.softmax(hypothesis, dim=1))).sum(dim=1).mean()
        
        optimizer.zero_grad()
        cost.backward()
        optimizer.step()
        
        if batch_idx == 31:
            print('Epoch {:4d}/{} Cost: {:.6f}'.format(
                epoch, nb_epochs, cost.item()
            ))

Epoch    0/10 Cost: 1.897210
Epoch    1/10 Cost: 1.562638
Epoch    2/10 Cost: 1.603787
Epoch    3/10 Cost: 1.521923
Epoch    4/10 Cost: 1.497116
Epoch    5/10 Cost: 1.495365
Epoch    6/10 Cost: 1.508261
Epoch    7/10 Cost: 1.495561
Epoch    8/10 Cost: 1.476127
Epoch    9/10 Cost: 1.530727
Epoch   10/10 Cost: 1.513487


In [10]:
hypothesis = F.softmax(xtrain.matmul(W) + b, dim=1) # or .mm or @
predict = torch.argmax(hypothesis, dim=1)

print(hypothesis)


# 정확도 계산 
correct_prediction = predict.float() == ytrain
print(correct_prediction)
accuracy = correct_prediction.sum().item() / len(correct_prediction)
print('The model has an accuracy of {:2.2f}% for the training set.'.format(accuracy * 100))

tensor([[4.7643e-15, 2.8324e-12, 5.5929e-16,  ..., 1.2421e-14, 2.3063e-05,
         9.3834e-15],
        [1.0000e+00, 7.4978e-27, 1.2517e-17,  ..., 1.3016e-20, 1.8036e-13,
         1.4631e-22],
        [5.9711e-14, 5.6302e-18, 6.3444e-12,  ..., 7.7315e-19, 6.8242e-09,
         1.1481e-12],
        ...,
        [2.5718e-23, 1.6715e-12, 2.1634e-18,  ..., 2.0472e-04, 4.7564e-06,
         9.9964e-01],
        [2.2130e-04, 1.3298e-18, 2.8798e-03,  ..., 2.6637e-13, 3.0843e-04,
         2.7360e-06],
        [3.4532e-09, 6.7802e-13, 2.2050e-08,  ..., 3.9469e-19, 1.0111e-04,
         1.3479e-08]], device='cuda:0', grad_fn=<SoftmaxBackward>)
tensor([ True,  True,  True,  ...,  True, False,  True], device='cuda:0')
The model has an accuracy of 94.06% for the training set.


In [11]:
xtest = torch.FloatTensor(np.array(test_data)).cuda()
hypothesis = F.softmax(xtest.matmul(W) + b, dim=1)
predict = torch.argmax(hypothesis, dim=1).cuda()
ytest=predict.detach().cpu().numpy()
submit_data['label']=ytest
print(submit_data.head())

   id  label
0   1      6
1   2      1
2   3      0
3   4      0
4   5      9


In [12]:
submit_data.to_csv("submission.csv",mode='w', index=False)